# Ensemble Learning

## Initial Imports

In [1]:
from _functions import *
from _functions import population_check
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Load the data
file_path = Path('Data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
print(df.isnull().sum())
df.head()

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
#Clean data for model; Convert issue date to numerical format
df['issue_month'] = df['issue_d'].str.split('-').str[0]
    
df["issue_month"] = df["issue_month"].apply(rename_months)
#Alternative Method, use get_dummies
#     df = pd.get_dummies(df, columns=['home_ownership','verification_status','initial_list_status','application_type'])

le = LabelEncoder()
# Encoding home_ownership column
le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])
# Encoding initial_list_status column
le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])
# Encoding initial_list_status column
le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])
# Encoding application_type column
le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])
#Format target variable
def loan_status(loan_status):
    if loan_status == "low_risk":
        return 0
    else:
        return 1
df["loan_status"] = df["loan_status"].apply(loan_status)
#Drop columns in which values are consistent for entire dataset. There is no value in using these features.
df.drop(columns=['pymnt_plan','hardship_flag','debt_settlement_flag','issue_d','next_pymnt_d'], inplace=True)
# Create our features
x_cols = [i for i in df.columns if i not in ('loan_status')]
X = df[x_cols]
# Create our target
y = df['loan_status']
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,issue_month
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,21.778153,0.217766,0.497697,12.587340,...,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,1.726172
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,20.199244,0.718367,0.758122,6.022869,...,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.743862
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,1.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,13.890000,0.000000,0.000000,8.000000,...,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,1.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,19.760000,0.000000,0.000000,11.000000,...,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,2.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,26.660000,0.000000,1.000000,16.000000,...,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,2.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,999.000000,18.000000,5.000000,72.000000,...,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,3.000000


In [ ]:
# population_check(X,y)

# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

In [ ]:
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Resample the training data with the BalancedRandomForestClassifier
clf_model = BalancedRandomForestClassifier(random_state=1)
clf_model = clf_model.fit(X_train_scaled,y_train)
y_predict = clf_model.predict(X_test_scaled)
results = pd.DataFrame({'Predictions': y_predict, 'Actual': y_test}).reset_index(drop=True)

# Calculated the balanced accuracy score
bas = balanced_accuracy_score(y_test,y_predict)

### Balanced Random Forest Classifier

In [ ]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_predict)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_predict)
# Displaying results
print("Confusion Matrix")
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_predict))

# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_predict))

Confusion Matrix
Accuracy Score : 0.8805579773321709
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.88      0.94     17118
           1       0.03      0.72      0.06        87

    accuracy                           0.88     17205
   macro avg       0.51      0.80      0.50     17205
weighted avg       0.99      0.88      0.93     17205

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.88      0.72      0.94      0.80      0.65     17118
          1       0.03      0.72      0.88      0.06      0.80      0.63        87

avg / total       0.99      0.88      0.72      0.93      0.80      0.65     17205



In [ ]:
# List the features sorted in descending order by feature importance
feature_importance = clf_model.feature_importances_
feature_importance_sorted = sorted(zip(clf_model.feature_importances_, X.columns), reverse=True)
feature_importance_sorted[:10]

[(0.06585662369230211, 'last_pymnt_amnt'),
 (0.062008346082494545, 'total_rec_prncp'),
 (0.056814057429300915, 'total_pymnt_inv'),
 (0.05339050684419236, 'total_rec_int'),
 (0.05289707638395531, 'total_pymnt'),
 (0.03419819064481047, 'issue_month'),
 (0.030655688608152216, 'int_rate'),
 (0.019342092192370475, 'dti'),
 (0.018539669026893753, 'out_prncp'),
 (0.01825105490320566, 'out_prncp_inv')]

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
eec_model = EasyEnsembleClassifier(random_state=1)
eec_model = eec_model.fit(X_train_scaled,y_train)
y_predict = eec_model.predict(X_test_scaled)
results = pd.DataFrame({'Predictions': y_predict, 'Actual': y_test}).reset_index(drop=True)
results.head(3)

AttributeError: 'EasyEnsembleClassifier' object has no attribute 'n_features_in_'

In [ ]:
# Calculated the balanced accuracy score
bas = balanced_accuracy_score(y_test,y_predict)

# Display the confusion matrix
# Calculating the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, y_predict)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_predict)

# Displaying results
print("Confusion Matrix")
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_predict))


# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_predict))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,14591,2527
Actual 1,15,72


Accuracy Score : 0.8522522522522522
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.85      0.92     17118
           1       0.03      0.83      0.05        87

    accuracy                           0.85     17205
   macro avg       0.51      0.84      0.49     17205
weighted avg       0.99      0.85      0.92     17205

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.85      0.83      0.92      0.84      0.71     17118
          1       0.03      0.83      0.85      0.05      0.84      0.70        87

avg / total       0.99      0.85      0.83      0.92      0.84      0.71     17205



### Final Questions
#
1. BalancedRandomForestClassifier had the best balanced accuracy score?
#
2. EasyEnsembleClassifier had the best recall score?
#
3. EasyEnsembleClassifier had the best geometric mean score?

#
4. The top three features Are:

    First: 'last_pymnt_amnt'.
    
    Second: 'total_rec_prncp'
    
    Third: 'total_pymnt_inv'